In [2]:
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
import os

from glob import glob
%matplotlib inline


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
test_video_dire=os.path.join('test_video')
test_img_dire=os.path.join('test_img')

In [6]:
model_dire=os.path.join('saved_model')

In [7]:
model = tf.saved_model.load(model_dire)
movenet = model.signatures['serving_default']

In [9]:
def get_keypoint_dict(outputs):
    outputs=outputs['output_0'].numpy()
    keypoints_dict={'Person_1':{},'Person_2':{},'Person_3':{},'Person_4':{},'Person_5':{},'Person_6':{}}
    for num,key in enumerate(keypoints_dict.keys()):
        keypoints_dict[key]['keypoints']=outputs[:,num,:51].reshape(17,3)
        keypoints_dict[key]['bbox']=outputs[:,num,51:].flatten()
    return keypoints_dict

    
    

In [10]:
def draw_keypoints(img,keypoints_dict,confidence):
    Y,X,c=img.shape
    for num,key in enumerate(keypoints_dict.keys()):
        for y,x,conf in keypoints_dict[key]['keypoints']:
            if conf>confidence:
                cv.circle(img,(int(X*x),int(Y*y)),6, (0,255,0), -1)
            
        

In [11]:
test_direlist=glob('test_video/*.mp4')

In [12]:
cap=cv.VideoCapture(test_direlist[0])
while cap.isOpened():
    success,img=cap.read()
    if success:
        input_img=img.copy()
        input_img=cv.resize(input_img,(608,320))
        input_img=np.array(input_img,dtype='int32')
        input_img=tf.convert_to_tensor(input_img)
        input_img=tf.expand_dims(input_img,axis=0)
        outputs=movenet(input_img)
        keypoints_dict=get_keypoint_dict(outputs)
        draw_keypoints(img,keypoints_dict,0.5)
        cv.imshow('demo',img)
        if cv.waitKey(1) & 0xFF==ord('q'):
            break
            
    else:
        break
cap.release()
cv.destroyAllWindows()
        

In [14]:
keypoints_dict['Person_1']['keypoints']

array([[0.43407735, 0.56814146, 0.40362287],
       [0.4265246 , 0.5724635 , 0.37499684],
       [0.42724872, 0.5649483 , 0.38669884],
       [0.43639207, 0.5795321 , 0.47017974],
       [0.43396917, 0.56123   , 0.49264583],
       [0.48082095, 0.5923663 , 0.69925755],
       [0.47790766, 0.5470916 , 0.5575337 ],
       [0.54386085, 0.60627586, 0.61809164],
       [0.5524576 , 0.5429514 , 0.3452951 ],
       [0.5999393 , 0.6048094 , 0.57454073],
       [0.62204844, 0.54288626, 0.21221794],
       [0.5982047 , 0.5842452 , 0.7860645 ],
       [0.59386677, 0.5558168 , 0.54132956],
       [0.67822164, 0.58935755, 0.722052  ],
       [0.6896928 , 0.55518883, 0.5013282 ],
       [0.7921428 , 0.5906784 , 0.6385105 ],
       [0.80043125, 0.56154203, 0.69905883]], dtype=float32)

In [23]:
test_direlist

['test_video\\C001100_003.mp4',
 'test_video\\C004200_005.mp4',
 'test_video\\C006200_005.mp4',
 'test_video\\C010100_002.mp4',
 'test_video\\C014100_005.mp4',
 'test_video\\C015200_001.mp4',
 'test_video\\C025200_001.mp4',
 'test_video\\C026100_001.mp4',
 'test_video\\clip_falldown_1.mp4',
 'test_video\\street-walking-falldown.mp4']

In [ ]:
a